# Modelos de Secuencias
### Intermedios

* Las tareas de prediccion de secuencias requiren que etiquetemos cada item en una secuencia
* Estas tareas son comunes en NLP:
    * _language modeling_: predecir la siguiente palabra dada una secuencia de palabras en cada paso.
    * _named entity recognition_: predecir si cada palabra es parte de una entidad nombrada como, persona, localizacion, producto o organizacion.

![sequence prediction tasks](../assets/seq-prediction-tasks.png)

**Dos Ejemplos de tareas de prediccion de secuencias**
* (a) _language modeling_ donde la tarea es predecir la siguiente palabra en una secuencia.
* (b) _named entity recognition_ donde la tarea es predecir los limites de de una entidad y su tipo de entidad.

## Las debilidades del Elman RNN

* Aunque el Elman RNN es adecuado para modelar secuencias, tiene dos problemas que lo hacen poco adecuado para muchas tareas:
    1. La inabilidad de retener informacion para predicciones a largo plazo.
    2. Estabilidad de las gradientes.
* Para entender estos dos problemos, recordemos que:
    * Los RNNs estan calculando un _hidden state vector_ en cada time step usando el _hidden state vector_ del time step anterior y el _input vector_ en el time step actual.
    * Este calculo hace al RNN bastante poderoso pero tambien crea problemas numericos.

### Retener informacion a largo plazo

* En cada time step estamos actualizando el _hidden state vector_ sin importar si hace sentido.
* Como consecuencia, el RNN no tien control sobre cuales valores se retienen y cuales se descartan en el _hidden state_
* Lo que deseamos es tener alguna forma de controlar:
    * si la actualizacion es opcional
    * o si deberia pasar,
    * por cuanto?
    * que partes?

### Problemas de estabilidad numerica

* El Elman RNN tiene una tiende a causar que las gradientes se salgan de control hacia cero o hacia infinito.
* Hay dos tipos de gradientes inestables que se salen de control:
    * _vanishing gradients_ cuando la direccion del valor absoluto de la gradiente se encoge
    * _exploding gradients_ cuando la direccion del valor absoluto de la gradiente se expande
* Cualquiera de estos dos problemas puede hacer el proceso de optimizacion inestable.

## Gating como una solucion a los problemas

* Para entender gating de forma intuitiva supongamos que estamos sumando dos cantidades, $a$ y $b$, pero queremos controlar cuanto de $b$ entra a la suma. Podemos reescribir la suma de $a + b$ como:

$$ a + \lambda b$$

donde $\lambda$ es un valor $[0, 1]$.
* Si $\lambda = 0$ entonces no hay ninguna contribucion de $b$
* Si $\lambda = 1$ entonces $b$ contribuye completamente

$\lambda$ esta actuando como un _switch_ o un _gate_ que controla la cantidad de $b$ que entra a la suma.

### Incorporando Gating al Elman RNN

Si el _hidden state_ previo era $h_{t-1}$ y el input actual es $x_t$, la actualizacion recurrente en el Elman RNN se veria algo asi:

$$ h_t = h_{t-1} + F(h_{t-1}, x_t)$$

donde,
* $F$ es el calculo recurrente del RNN

* Ahora supongamos que, en vez de ser un constante, el $\lambda$ en el ejemplo previo era una funcion de: 
    * _hidden state vector_ $h_{t-1}$ y 
    * el input actual $x_t$, y produjera el comportamiento deseado de _gating_.
* Es decir, un valor $[0,1]$
* Con esta funcion de _gating_, la ecuacion para la actualizacion del RNN seria la siguiente:

$$ h_t = h_{t-1} + \lambda(h_{t-1}, x_t)F(h_{t-1}, x_t)$$

* Podemos ver que la funcion $\lambda$ controla cuanto de del input actual puede actualizar es estado $h_{t-1}$
* La funcion $\lambda$ es usualmente una funcion sigmoide que sabemos produce un valor entre $[0,1]$

## LSTM

* En el caso de una red _long short-term memory_ la intuicion basica se extiende para incorporar no solo actualizaciones condicionadas, pero tambien olvido intencional de los valores en el _hidden state_ previo $h_{t-1}$.
* Esta funcion de _forgetting_ sucede multiplicando el _hidden state_ previo $h_{t-1}$ por otra funcion, $\mu$, que tambien produce un valor entre $[0,1]$ y depende del input actual:

$$ h_t = \mu(h{t-1}, x_t)h_{t-1} + \lambda(h_{t-1}, x_t)F(h_{t-1}, x_t)$$

* $\mu$ es otra funcion de _gating_
* En una descripcion real del LSTM, se vuelve un poco mas complicado porque las _gating functions_ son parametrizadas, lo que lleva a una secuencia de operaciones un poco complejas.
* Para conocer mas sobre LSTMs lean el [articulo clasico por Christopher Olah](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

* El LSTM es una de las muchas variaciones con _gates_ del RNN. 
* Otra de las mas populares es el _gated recurrent unit_ GRU
* En PyTorch simplemente se puede reemplazar el `nn.RNN` o `nn.RNNCell` con un `nn.LSTM` o `nn.LSTMCell`
* Igual para un `nn.GRU`